In [1]:
import os
import sys
import time
import random
import numpy as np
import tensorflow as tf
import pickle

import skimage
import skimage.io
import skimage.transform
from matplotlib.pyplot import imshow
%matplotlib inline

from PIL import Image
import scipy
from sklearn.utils import shuffle

from sklearn.metrics import accuracy_score

from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def leaky_relu(features, alpha=0.2, name=None):
    with ops.name_scope(name, "LeakyRelu", [features, alpha]):
        features = ops.convert_to_tensor(features, name="features")
        alpha = ops.convert_to_tensor(alpha, name="alpha")
        return math_ops.maximum(alpha * features, features)

## argument setting

In [2]:
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

img_path = './data'
len_img_all = 33431
condition = True
load_data = True
tag_img_only = True
batch_size = 16
iteration = 100000
D_ITER = 1
G_ITER = 1
d_lda = 10

toy_test = False
if toy_test :
    len_img_all = 512
    load_data = False
if tag_img_only :
    len_img_all = 11568
output_str = 'gan'
load_model = False

pretrain_hair = True
pretrain_eyes = True

## preprocessing

In [3]:
def img_prep(save=False, lst_null=None) :
    print ('img_prep is runing')
    if tag_img_only :
        lst_img = []
        for img_i in range(len_img_all) :
            if img_i in lst_null :
                continue
            img_temp = skimage.io.imread('{}/faces/{}.jpg'.format(img_path,img_i))
            img_temp = skimage.transform.resize(img_temp, (64,64))
            lst_img += [img_temp]
    else :
        ary_img = np.zeros((len_img_all,64,64,3))
        for img_i in range(len_img_all) :
            img_temp = skimage.io.imread('{}/faces/{}.jpg'.format(img_path,img_i))
            img_temp = skimage.transform.resize(img_temp, (64,64))
            ary_img[img_i] = img_temp
    if save :
        if tag_img_only :
            print ('./data_pp/ary_img_withtag is saving')
            ary_img = np.asarray(lst_img)
            np.save('./data_pp/ary_img_withtag', ary_img)
        else :
            np.save('./data_pp/ary_img', ary_img)
            
    print ('len_ary_img : {}'.format(len(ary_img)))
    return ary_img
    
def tag_prep(save=False) :
    # 1,6
    # 2,4,12
    # 3,5,9
    # 7,10
    lst_hair = ['null', 'orange hair', 'white hair', 'aqua hair', 'gray hair',
                'green hair', 'red hair', 'purple hair', 'pink hair',
                'blue hair', 'black hair', 'brown hair', 'blonde hair']
    lst_eyes = ['null', 'gray eyes', 'black eyes', 'orange eyes',
                'pink eyes', 'yellow eyes', 'aqua eyes', 'purple eyes',
                'green eyes', 'brown eyes', 'red eyes', 'blue eyes']
    
    h_n = 0
    e_n = 0  
    lst_null = []
    if tag_img_only :
        lst_tag_hair = []
        lst_tag_eyes = []
        with open('{}/tags_clean.csv'.format(img_path), 'r') as f :
            for line_i,line in enumerate(f.readlines()[:len_img_all]) :
        #         sys.stdout.write("\r{}\t".format(line_i))
        #         sys.stdout.flush()

                FLAG_h = False
                FLAG_e = False
                FLAG_double_tag = False
                for h_i,h in enumerate(lst_hair) :
                    if h in line :
                        if FLAG_h :
                            # double tag
                            lst_tag_hair = lst_tag_hair[:-1]
                            lst_null += [line_i]
                            FLAG_double_tag = True
                            break
                        FLAG_h = 1
                        lst_tag_hair += [h_i]
                        h_n += 1
                if not FLAG_h :
                    lst_null += [line_i]
                    continue
                if FLAG_double_tag :
                    continue
                for e_i,e in enumerate(lst_eyes) :
                    if e in line :
                        if FLAG_e :
                            # double tag
                            lst_tag_eyes = lst_tag_eyes[:-1]
                            lst_null += [line_i]
                            lst_tag_hair = lst_tag_hair[:-1]
                            break
                        FLAG_e = 1
                        lst_tag_eyes += [e_i]
                        e_n += 1
                if not FLAG_e :
                    lst_null += [line_i]
                    lst_tag_hair = lst_tag_hair[:-1]
            print (len(lst_tag_hair))
            print (len(lst_tag_eyes))
            print ('number of hair tag : {}'.format(h_n))
            print ('number of eyes tag : {}'.format(e_n))
            print ('number of total img : {}'.format(line_i+1))
            print ('number of total both tag : {}'.format(len(lst_tag_hair)))
            print ('number of drop tag : {}'.format(len(lst_null)))
            assert len(lst_tag_hair) == len(lst_tag_eyes), 'error'
            print ('tag preprocessing done')
            
    else :
        ary_tag_hair = np.zeros((len_img_all))
        ary_tag_eyes = np.zeros((len_img_all))
        with open('{}/tags_clean.csv'.format(img_path), 'r') as f :
            for line_i,line in enumerate(f.readlines()[:len_img_all]) :
        #         sys.stdout.write("\r{}\t".format(line_i))
        #         sys.stdout.flush()

                flag_h = 0
                flag_e = 0
                for h_i,h in enumerate(lst_hair) :
                    if h in line :
                        ary_tag_hair[line_i] = h_i
                        h_n += 1
                for e_i,e in enumerate(lst_eyes) :
                    if e in line :
                        ary_tag_eyes[line_i] = e_i
                        e_n += 1
            assert len(ary_tag_hair) == len(ary_tag_eyes), 'error'
            print ('number of hair tag : {}'.format(h_n))
            print ('number of eyes tag : {}'.format(e_n))
            print ('number of total img : {}'.format(line_i))
            print ('tag preprocessing done')
        
    if save :
        if not os.path.isdir('./data_pp') :
            os.makedirs('./data_pp')
        if tag_img_only :
            print ('./data_pp/ary_tag_hair_withtag is saving')
            ary_tag_hair = np.asarray(lst_tag_hair)
            ary_tag_eyes = np.asarray(lst_tag_eyes)
            np.save('./data_pp/ary_tag_hair_withtag', ary_tag_hair)
            np.save('./data_pp/ary_tag_eyes_withtag', ary_tag_eyes)
        else :
            print ('./data_pp/ary_tag_hair is saving')
            np.save('./data_pp/ary_tag_hair', ary_tag_hair)
            np.save('./data_pp/ary_tag_eyes', ary_tag_eyes)
    
    return ary_tag_hair, ary_tag_eyes, lst_null
    
def preprocess(show=True, save=False, load=False) :
    if load :
        if tag_img_only :
            ary_tag_hair = np.load('./data_pp/ary_tag_hair_withtag.npy')
            ary_tag_eyes = np.load('./data_pp/ary_tag_eyes_withtag.npy')
            ary_img = np.load('./data_pp/ary_img_withtag.npy')
        else :
            ary_tag_hair = np.load('./data_pp/ary_tag_hair.npy')
            ary_tag_eyes = np.load('./data_pp/ary_tag_eyes.npy')
            ary_img = np.load('./data_pp/ary_img.npy')
    else :
        ary_tag_hair, ary_tag_eyes, lst_null = tag_prep(save=save)
        ary_img = img_prep(save=save, lst_null=lst_null)

    if show :
        print ('ary_tag_hair shape : {}'.format(ary_tag_hair.shape))
        print ('ary_tag_eyes shape : {}'.format(ary_tag_eyes.shape))
        print ('ary_img shape : {}'.format(ary_img.shape))
#         imshow(ary_img[1])
    
    return ary_tag_hair, ary_tag_eyes, ary_img

ary_tag_hair, ary_tag_eyes, ary_img = preprocess(show=True,save=False,load=load_data)

ary_tag_hair shape : (11568,)
ary_tag_eyes shape : (11568,)
ary_img shape : (11568, 64, 64, 3)


## model

In [ ]:
class GAN(object):
    def __init__(self) :
        self.lr = 0.0001
        self.lr_pre = 0.0001
        self.momentum = 0.5
        self.bs = batch_size # batch size is m of paper
        self.bs_pre = 128
        self.epoch = 10000
        self.hair_n = 13
        self.eyes_n = 12
        self.lda = d_lda
        self.epsilon = 0.5
        self.activation = leaky_relu
        self.initializer = tf.contrib.keras.initializers.he_normal()
            
    def build_G_net(self) :
        with tf.variable_scope('G') as g_scope:
            self.G_in_hair = tf.placeholder(tf.int32, shape=[None])
            self.G_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.G_in_noise = tf.placeholder(tf.float32, shape=[None,100])
            
            self.G_H_onehot = tf.one_hot(self.G_in_hair, self.hair_n)
            self.G_E_onehot = tf.one_hot(self.G_in_eyes, self.eyes_n)
            g = tf.concat([self.G_H_onehot, self.G_E_onehot, self.G_in_noise], axis=1)
            g = tf.layers.dense(g,4*4*1024,activation=None)
            g = tf.reshape(g,(-1,4,4,1024))
            
            g = tf.layers.conv2d_transpose(g, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.batch_normalization(g)
            g = tf.layers.conv2d_transpose(g, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.batch_normalization(g)
            g = tf.layers.conv2d_transpose(g, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=self.activation)
            g = tf.layers.batch_normalization(g)
            self.g = tf.layers.conv2d_transpose(g, filters=3, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                           padding='same', activation=tf.nn.tanh)
            self.img = (self.g + 1.0) / 2.0
        
    def build_D_net(self) :

        with tf.variable_scope('D') as d_scope:

            ### include right and fake1
            self.D_in_hair = tf.placeholder(tf.int32, shape=[None])
            self.D_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.D_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3])
#             self.D_in_img = self.D_in_img*2 - 1
            
            D_H_onehot = tf.one_hot(self.D_in_hair, self.hair_n)
            D_E_onehot = tf.one_hot(self.D_in_eyes, self.eyes_n)
            
            self.D_in_hair_right = D_H_onehot[:self.bs]
            self.D_in_eyes_right = D_E_onehot[:self.bs]
            self.D_in_img_right = self.D_in_img[:self.bs]
            
            self.D_in_hair_fake = D_H_onehot[self.bs:]
            self.D_in_eyes_fake = D_E_onehot[self.bs:]
            self.D_in_img_fake = self.D_in_img[self.bs:]
            
            use_epsilon_uniform = 1
            if use_epsilon_uniform :
                self.epsilon = tf.placeholder(tf.float32, shape=[1])
                self.D_x_hat_hair = self.epsilon*self.D_in_hair_right + (1-self.epsilon)*self.D_in_hair_fake
                self.D_x_hat_eyes = self.epsilon*self.D_in_eyes_right + (1-self.epsilon)*self.D_in_eyes_fake
                self.D_x_hat_img = self.epsilon*self.D_in_img_right + (1-self.epsilon)*self.D_in_img_fake
            else :
                self.D_x_hat_hair = self.epsilon*self.D_in_hair_right + (1-self.epsilon)*self.D_in_hair_fake
                self.D_x_hat_eyes = self.epsilon*self.D_in_eyes_right + (1-self.epsilon)*self.D_in_eyes_fake
                self.D_x_hat_img = self.epsilon*self.D_in_img_right + (1-self.epsilon)*self.D_in_img_fake
                
            self.D_img = tf.concat([self.D_in_img_right,self.D_in_img_fake,self.D_x_hat_img], axis=0)
            self.D_hair = tf.concat([self.D_in_hair_right,self.D_in_hair_fake,self.D_x_hat_hair], axis=0)
            self.D_eyes = tf.concat([self.D_in_eyes_right,self.D_in_eyes_fake,self.D_x_hat_eyes], axis=0)
            
            d = tf.layers.conv2d(self.D_img, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c1', reuse=None)
            d = tf.layers.conv2d(d, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c2', reuse=None)
            d = tf.layers.conv2d(d, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c3', reuse=None)
            d = tf.layers.conv2d(d, filters=1024, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c4', reuse=None)
            
            D_tag_in = tf.concat([self.D_hair, self.D_eyes], axis=1)
            D_tag_in = tf.layers.dense(D_tag_in,4*4*1024,activation=self.activation, name='d_d1', kernel_initializer=self.initializer, reuse=None)
            D_tag_in = tf.reshape(D_tag_in,(-1,4,4,1024))
            
            d = tf.concat([d, D_tag_in], axis=3)
            d = tf.layers.conv2d(d, filters=256, kernel_size=(1,1), strides=(1,1), 
                                 padding='same', activation=self.activation, name='d_c5', kernel_initializer=self.initializer, reuse=None)
            d = tf.reshape(d, [-1, 4*4*256]) 
            self.d_temp = tf.layers.dense(d, 1 ,activation=None, name='d_d2', kernel_initializer=self.initializer, reuse=None)
        
            self.d_right, self.d_fake, self.d_x_hat = tf.split(self.d_temp, num_or_size_splits=3, axis=0)
#             self.d_right, self.d_fake_temp = tf.split(self.d_temp, num_or_size_splits=2, axis=0)
#             self.d_fake, self.d_x_hat = tf.split(self.d_fake_temp, num_or_size_splits=2, axis=0)
            
            ### gradient penalty
#             gradient_temp = tf.gradients(self.d_x_hat,[self.D_x_hat_img,self.D_x_hat_hair,self.D_x_hat_eyes])
            gradient_img_temp = tf.gradients(self.d_x_hat,self.D_x_hat_img)
            gradient_hair_temp = tf.gradients(self.d_x_hat,self.D_x_hat_hair)
            gradient_eyes_temp = tf.gradients(self.d_x_hat,self.D_x_hat_eyes)
#             self.gradient_penalty = tf.maximum(tf.constant(0.0, shape=[self.bs,1]), tf.sqrt(tf.square(gradient_temp)) - tf.constant(1.0, shape=[self.bs,1]))
            self.gradient_penalty_img = tf.square(tf.sqrt(tf.reduce_sum(tf.square(gradient_img_temp), axis=1)) - 1)
#             self.gradient_penalty_img = self.lda * tf.reduce_mean(self.gradient_penalty_img)
            self.gradient_penalty_hair = tf.square(tf.sqrt(tf.reduce_sum(tf.square(gradient_hair_temp), axis=1)) - 1)
#             self.gradient_penalty_hair = self.lda * tf.reduce_mean(self.gradient_penalty_hair)
            self.gradient_penalty_eyes = tf.square(tf.sqrt(tf.reduce_sum(tf.square(gradient_eyes_temp), axis=1)) - 1)
#             self.gradient_penalty_eyes = self.lda * tf.reduce_mean(self.gradient_penalty_eyes)
            self.gradient_penalty = self.lda * (tf.reduce_mean(self.gradient_penalty_img)
                                                + tf.reduce_mean(self.gradient_penalty_hair)/10
                                                + tf.reduce_mean(self.gradient_penalty_eyes)/10)
            
            ### final loss
            self.d_loss = -(tf.reduce_mean(self.d_right) - tf.reduce_mean(self.d_fake) - self.gradient_penalty)
            self.d_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"D/")
            self.d_train = tf.train.AdamOptimizer(learning_rate=self.lr,beta1=0.0,beta2=0.9).minimize(self.d_loss, var_list=self.d_train_vars)
            
            ### G_D loss
            self.G_D_in_hair = self.G_in_hair
            self.G_D_in_eyes = self.G_in_eyes
            self.G_D_in_img = self.g
            
            self.G_D_eyes = tf.one_hot(self.G_D_in_eyes, self.eyes_n)
            self.G_D_hair = tf.one_hot(self.G_D_in_hair, self.hair_n)
            
            g_d = tf.layers.conv2d(self.G_D_in_img, filters=128, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c1', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=256, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c2', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=512, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c3', reuse=True)
            g_d = tf.layers.conv2d(g_d, filters=1024, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='d_c4', reuse=True)
            
            G_D_tag_in = tf.concat([self.G_D_hair, self.G_D_eyes], axis=1)
            G_D_tag_in = tf.layers.dense(G_D_tag_in,4*4*1024,activation=self.activation, name='d_d1', kernel_initializer=self.initializer, reuse=True)
            G_D_tag_in = tf.reshape(G_D_tag_in,(-1,4,4,1024))
            
            g_d = tf.concat([g_d, G_D_tag_in], axis=3)
            g_d = tf.layers.conv2d(g_d, filters=256, kernel_size=(1,1), strides=(1,1), 
                                 padding='same', activation=self.activation, name='d_c5', kernel_initializer=self.initializer, reuse=True)
            g_d = tf.reshape(g_d, [-1, 4*4*256]) 
            self.g_d_temp = tf.layers.dense(g_d, 1 ,activation=None, name='d_d2', kernel_initializer=self.initializer, reuse=True)
            
            self.gd_loss = - tf.reduce_mean(self.g_d_temp)
            
            #
            # HE classification
            #
            
            # hair classification pretrained
            self.hair_in_hair = tf.placeholder(tf.int32, shape=[None])
            self.hair_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3]) 

            hair_H_onehot = tf.one_hot(self.hair_in_hair, self.hair_n)
            hair_temp = tf.layers.batch_normalization(self.hair_in_img, name='hair_b0', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c1', reuse=None)
            hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b1', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c2', reuse=None)
            hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b2', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c3', reuse=None)
            hair_temp = tf.layers.batch_normalization(hair_temp, name='hair_b3', reuse=None)
            hair_temp = tf.layers.conv2d(hair_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c4', reuse=None)
            hair_temp = tf.reshape(hair_temp, [-1, 4*4*32]) 
            self.hair = tf.layers.dense(hair_temp, self.hair_n ,activation=tf.nn.softmax, name='hair_s2', kernel_initializer=self.initializer, reuse=None)

            self.hair_loss = tf.reduce_mean(-tf.reduce_sum(hair_H_onehot * tf.log(self.hair), axis=1))
            self.hair_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"D/")
            self.hair_train = tf.train.AdamOptimizer(learning_rate=self.lr_pre).minimize(self.hair_loss, var_list=self.hair_train_vars)
            
            # eyes classification pretrained
            self.eyes_in_eyes = tf.placeholder(tf.int32, shape=[None])
            self.eyes_in_img = tf.placeholder(tf.float32, shape=[None,64,64,3]) 

            eyes_E_onehot = tf.one_hot(self.eyes_in_eyes, self.eyes_n)
            eyes_temp = tf.layers.batch_normalization(self.eyes_in_img, name='eyes_b0', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c1', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b1', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c2', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b2', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c3', reuse=None)
            eyes_temp = tf.layers.batch_normalization(eyes_temp, name='eyes_b3', reuse=None)
            eyes_temp = tf.layers.conv2d(eyes_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c4', reuse=None)
            eyes_temp = tf.reshape(eyes_temp, [-1, 4*4*32]) 
            self.eyes = tf.layers.dense(eyes_temp, self.eyes_n ,activation=tf.nn.softmax, name='eyes_s2', kernel_initializer=self.initializer, reuse=None)

            self.eyes_loss = tf.reduce_mean(-tf.reduce_sum(eyes_E_onehot * tf.log(self.eyes), axis=1))
            self.eyes_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"D/")
            self.eyes_train = tf.train.AdamOptimizer(learning_rate=self.lr_pre).minimize(self.eyes_loss, var_list=self.eyes_train_vars)
            
            ### hair classification
            gh_temp = tf.layers.batch_normalization(self.g, name='hair_b0', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c1', reuse=True)
            gh_temp = tf.layers.batch_normalization(gh_temp, name='hair_b1', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c2', reuse=True)
            gh_temp = tf.layers.batch_normalization(gh_temp, name='hair_b2', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c3', reuse=True)
            gh_temp = tf.layers.batch_normalization(gh_temp, name='hair_b3', reuse=True)
            gh_temp = tf.layers.conv2d(gh_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='hair_c4', reuse=True)
            gh_temp = tf.reshape(gh_temp, [-1, 4*4*32]) 
            self.gh = tf.layers.dense(gh_temp, self.hair_n ,activation=tf.nn.softmax, name='hair_s2', kernel_initializer=self.initializer, reuse=True)
            
            self.gh_loss = tf.reduce_mean(-tf.reduce_sum(self.G_H_onehot * tf.log(self.gh), axis=1))
            
            ### eyes classification
            ge_temp = tf.layers.batch_normalization(self.g, name='eyes_b0', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c1', reuse=True)
            ge_temp = tf.layers.batch_normalization(ge_temp, name='eyes_b1', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c2', reuse=True)
            ge_temp = tf.layers.batch_normalization(ge_temp, name='eyes_b2', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c3', reuse=True)
            ge_temp = tf.layers.batch_normalization(ge_temp, name='eyes_b3', reuse=True)
            ge_temp = tf.layers.conv2d(ge_temp, filters=32, kernel_size=(5,5), strides=(2,2), kernel_initializer=self.initializer,
                                 padding='same', activation=self.activation, name='eyes_c4', reuse=True)
            ge_temp = tf.reshape(ge_temp, [-1, 4*4*32]) 
            self.ge = tf.layers.dense(ge_temp, self.eyes_n ,activation=tf.nn.softmax, name='eyes_s2', kernel_initializer=self.initializer, reuse=True)

            self.ge_loss = tf.reduce_mean(-tf.reduce_sum(self.G_E_onehot * tf.log(self.ge), axis=1))
            
            
            
            self.gd_loss_final = 1.*self.gd_loss + 1.*2.*self.gh_loss + 1.*5.*self.ge_loss
            
            self.gd_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"G/")
            self.gd_train = tf.train.AdamOptimizer(learning_rate=self.lr,beta1=0.0,beta2=0.9).minimize(self.gd_loss_final, var_list=self.gd_train_vars)
            
    def build_net(self) :
#         self.build_hair_net()
#         self.build_eyes_net()
        
        self.build_G_net()
        self.build_D_net()
        
        self.sess = tf.Session(config=config)
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        self.sum_img = tf.summary.image('image',self.img, max_outputs=5)
        self.sum_D_loss = tf.summary.scalar('D_loss',self.d_loss)
        self.sum_G_loss = tf.summary.scalar('G_loss',self.gd_loss_final)
        self.sum_merge = tf.summary.merge([self.sum_img, self.sum_D_loss, self.sum_G_loss]) 
        
        self.writer = tf.summary.FileWriter("logs/", self.sess.graph)
#         self.writer = tf.summary.FileWriter("logs_temp/", tf.get_default_graph())
        self.writer.flush()


## training

In [ ]:
def training() :
    tf.reset_default_graph()
    ary_tag_hair, ary_tag_eyes, ary_img = preprocess(save=False,load=load_data)
    if condition :
        gan = GAN()
        gan.build_net()
        if load_model :
            pass
        pretrain_hair = True
        if pretrain_hair :
            for i in range(14) :
                print (i)
                ary_tag_hair_sh, ary_tag_eyes_sh, ary_img_sh = shuffle(ary_tag_hair[:-1000], ary_tag_eyes[:-1000], ary_img[:-1000], random_state=i)
                ary_img_sh = ary_img_sh*2 -1
                b_i = 0
                while b_i+gan.bs_pre < len_img_all-1000 :
                    sys.stdout.write("\r{}\t".format(b_i))
                    sys.stdout.flush()
                    b_tag_hair = ary_tag_hair_sh[b_i:b_i+gan.bs_pre]
                    b_img = ary_img_sh[b_i:b_i+gan.bs_pre]
                    
                    _, loss = gan.sess.run([gan.hair_train,gan.hair_loss], feed_dict={gan.hair_in_hair:b_tag_hair,
                                                            gan.hair_in_img:b_img})
#                     print (loss)
                    b_i += gan.bs_pre
                ### validation    
                y_pred = np.argmax(gan.sess.run(gan.hair, feed_dict={gan.hair_in_img:ary_img[-500:]}), axis=1)
                y_true = ary_tag_hair[-500:]
#                 print ('\n')
#                 print (y_pred)
#                 print (y_true)
                print ('acc : {}'.format(accuracy_score(y_true,y_pred)))

        pretrain_eyes = True
        if pretrain_eyes :
            for i in range(14) :
                print (i)
                ary_tag_eyes_sh, ary_tag_eyes_sh, ary_img_sh = shuffle(ary_tag_eyes[:-1000], ary_tag_eyes[:-1000], ary_img[:-1000], random_state=i)
                ary_img_sh = ary_img_sh*2 -1
                b_i = 0
                while b_i+gan.bs_pre < len_img_all-1000 :
                    sys.stdout.write("\r{}\t".format(b_i))
                    sys.stdout.flush()
                    b_tag_eyes = ary_tag_eyes_sh[b_i:b_i+gan.bs_pre]
                    b_img = ary_img_sh[b_i:b_i+gan.bs_pre]
                    
                    _, loss = gan.sess.run([gan.eyes_train,gan.eyes_loss], feed_dict={gan.eyes_in_eyes:b_tag_eyes,
                                                            gan.eyes_in_img:b_img})
#                     print (loss)
                    b_i += gan.bs_pre
                ### validation    
                y_pred = np.argmax(gan.sess.run(gan.eyes, feed_dict={gan.eyes_in_img:ary_img[-500:]}), axis=1)
                y_true = ary_tag_eyes[-500:]
#                 print ('\n')
#                 print (y_pred)
#                 print (y_true)
                print ('acc : {}'.format(accuracy_score(y_true,y_pred)))
    else :
        gan = GAN_no_condition()
        gan.build_net()
    
    lst_loss_his_d = []
    lst_loss_his_g = []
    for i in range(iteration) :
        print(i)
        ary_tag_hair_sh, ary_tag_eyes_sh, ary_img_sh = shuffle(ary_tag_hair, ary_tag_eyes, ary_img, random_state=i)
        gen_tag_hair = np.random.randint(13, size=int(len_img_all/2+1))
        gen_tag_eyes = np.random.randint(12, size=int(len_img_all/2+1))
        ary_img_sh = ary_img_sh*2 -1
        
        b_i = 0
        while b_i+gan.bs <= len_img_all :
            b_tag_hair_right = ary_tag_hair_sh[b_i:b_i+gan.bs]
            b_tag_eyes_right = ary_tag_eyes_sh[b_i:b_i+gan.bs]
            b_img_right = ary_img_sh[b_i:b_i+gan.bs]
            
            # fake1 (right img wrong text)
            b_tag_hair_fake1 = np.random.randint(1,13, size=int(gan.bs/4))
            b_tag_eyes_fake1 = np.random.randint(1,12, size=int(gan.bs/4))
            b_img_fake1 = np.copy(b_img_right[:int(gan.bs/4)])
            for ii in range(int(gan.bs/4)) :
                while b_tag_hair_fake1[ii] == b_tag_hair_right[ii] :
                    b_tag_hair_fake1[ii] = random.randint(0,13)
                while b_tag_eyes_fake1[ii] == b_tag_eyes_right[ii] :
                    b_tag_eyes_fake1[ii] = random.randint(0,12)
                    
            # fake2 (wrong img right text)
            b_tag_hair_fake2 = np.copy(b_tag_hair_right[:int(gan.bs/4)])
            b_tag_eyes_fake2 = np.copy(b_tag_eyes_right[:int(gan.bs/4)])
            lst_random_num = random.sample(range(len_img_all),k=int(gan.bs/4))
            b_img_fake2 = np.copy(ary_img_sh[lst_random_num])
            
            # fake3 (generate img right text)
            b_tag_hair_fake3 = gen_tag_hair[int(b_i/2):int(b_i/2+gan.bs/2)]
            b_tag_eyes_fake3 = gen_tag_eyes[int(b_i/2):int(b_i/2+gan.bs/2)]
            ary_temp = np.random.normal(0,1,[b_tag_eyes_fake3.shape[0],100])

            b_img_fake3 = gan.sess.run(gan.g, feed_dict={gan.G_in_hair:b_tag_hair_fake3, 
                                                         gan.G_in_eyes:b_tag_eyes_fake3, 
                                                         gan.G_in_noise:ary_temp})
            
            
            # update D
            b_tag_hair = np.concatenate((b_tag_hair_right,b_tag_hair_fake1,b_tag_hair_fake2,b_tag_hair_fake3), axis=0)
            b_tag_eyes = np.concatenate((b_tag_eyes_right,b_tag_eyes_fake1,b_tag_eyes_fake2,b_tag_eyes_fake3), axis=0)
            b_img = np.concatenate((b_img_right,b_img_fake1,b_img_fake2,b_img_fake3), axis=0)
            b_epsilon = np.random.rand(1,)
            for i2 in range(D_ITER) :
                _, loss_D = gan.sess.run([gan.d_train,gan.d_loss], feed_dict={gan.D_in_hair:b_tag_hair, 
                                                          gan.D_in_eyes:b_tag_eyes, 
                                                          gan.D_in_img:b_img,
                                                          gan.epsilon:b_epsilon})
                tf.summary.scalar('loss_D', loss_D)
            
                print ('D loss : {}'.format(loss_D))
            
            # update G
            b_tag_hair_g = gen_tag_hair[int(b_i/2):int(b_i/2+gan.bs/2)]
            b_tag_eyes_g = gen_tag_eyes[int(b_i/2):int(b_i/2+gan.bs/2)]
            ary_temp_g = np.random.normal(0,1,[b_tag_hair_g.shape[0],100])
            for i3 in range(G_ITER) :
                _, loss_G = gan.sess.run([gan.gd_train,gan.gd_loss], feed_dict={gan.G_in_hair:b_tag_hair_g, 
                                                         gan.G_in_eyes:b_tag_eyes_g, 
                                                         gan.G_in_noise:ary_temp_g})
    
                print ('G loss : {}'.format(loss_G))
            
            
            if b_i % 6400 == 0 :
                lst_loss_his_d += [loss_D]
                lst_loss_his_g += [loss_G]
                print ('saving model...')
                if not os.path.isdir('./model_tf') :
                    os.mkdir('./model_tf')
                if not os.path.isdir('./record') :
                    os.mkdir('./record')
                if not os.path.isdir('./img') :
                    os.mkdir('./img')
                k = 0
                while 1 :
                    if os.path.isfile('./model_tf/model_{}_{}.ckpt.meta'.format(output_str, k)) :
                        k += 1
                    else :
                        break
                save_path = gan.saver.save(gan.sess, './model_tf/model_{}_{}.ckpt'.format(output_str, k))
                with open('./record/loss_g_{}_{}.pkl'.format(output_str, k), 'wb') as f:
                    pickle.dump(lst_loss_his_g, f)
                with open('./record/loss_d_{}_{}.pkl'.format(output_str, k), 'wb') as f:
                    pickle.dump(lst_loss_his_d, f)
                img_sample = gan.sess.run(gan.g, feed_dict={gan.G_in_hair:b_tag_hair_fake3, 
                                                         gan.G_in_eyes:b_tag_eyes_fake3, 
                                                         gan.G_in_noise:ary_temp})
                scipy.misc.imsave('img/img_sample_{}_{}.jpg'.format(output_str,k),img_sample[0])
                summary = gan.sess.run(gan.sum_merge, feed_dict={gan.G_in_hair:b_tag_hair_fake3, 
                                                         gan.G_in_eyes:b_tag_eyes_fake3, 
                                                         gan.G_in_noise:ary_temp,
                                                         gan.D_in_hair:b_tag_hair, 
                                                         gan.D_in_eyes:b_tag_eyes, 
                                                         gan.D_in_img:b_img,
                                                         gan.epsilon:b_epsilon})
                gan.writer.add_summary(summary)
                gan.writer.flush()
                print("Model saved in file: %s" % save_path)
    
            b_i += gan.bs
        
training()    

ary_tag_hair shape : (11568,)
ary_tag_eyes shape : (11568,)
ary_img shape : (11568, 64, 64, 3)
0
10368	acc : 0.28
1
10368	acc : 0.318
2
10368	acc : 0.372
3
10368	acc : 0.426
4
10368	acc : 0.386
5
10368	acc : 0.41
6
10368	acc : 0.448
7
10368	acc : 0.432
8
10368	acc : 0.442
9
10368	acc : 0.428
10
10368	acc : 0.474
11
10368	acc : 0.454
12
10368	acc : 0.464
13
10368	acc : 0.474
0
10368	acc : 0.25
1
10368	acc : 0.254
2
10368	acc : 0.326
3
10368	acc : 0.34
4
10368	acc : 0.338
5
10368	acc : 0.388
6
10368	acc : 0.398
7
10368	acc : 0.408
8
10368	acc : 0.42
9
10368	acc : 0.42
10
10368	acc : 0.46
11
10368	acc : 0.438
12
10368	acc : 0.476
13
10368	acc : 0.45
0
D loss : 10.172513961791992
G loss : -0.62687087059021
saving model...


/nfs/nas-5.1/thchu/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:153: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Model saved in file: ./model_tf/model_gan_9.ckpt
D loss : 8.417336463928223
G loss : -1.3426309823989868
D loss : 5.490331172943115
G loss : -2.581075668334961
D loss : -0.5053272247314453
G loss : -7.246461868286133
D loss : -12.79457950592041
G loss : -18.74063491821289
D loss : -17.920761108398438
G loss : -51.95989990234375
D loss : -41.748538970947266
G loss : -132.4843292236328
D loss : -38.688720703125
G loss : -212.2857666015625
D loss : -23.410396575927734
G loss : -195.1951904296875
D loss : -43.0588493347168
G loss : -203.06643676757812
D loss : -79.8524398803711
G loss : -221.112548828125
D loss : -67.7734603881836
G loss : -303.2996826171875
D loss : -51.983680725097656
G loss : -212.07675170898438
D loss : -85.73978424072266
G loss : -206.914306640625
D loss : -221.46945190429688
G loss : -115.89527130126953
D loss : -232.73602294921875
G loss : -17.437734603881836
D loss : -187.38241577148438
G loss : -2.408881187438965
D loss : -198.63458251953125
G loss : 50.6571731567

G loss : 13568.048828125
D loss : -2085.5556640625
G loss : 8377.134765625
D loss : -2070.398681640625
G loss : 8947.650390625
D loss : 164.0377197265625
G loss : 2963.11181640625
D loss : 359.79876708984375
G loss : -1754.6671142578125
D loss : -2171.49658203125
G loss : -6849.373046875
D loss : -831.978759765625
G loss : -4771.26708984375
D loss : -2110.390625
G loss : -5484.72119140625
D loss : -666.4263305664062
G loss : -3406.314208984375
D loss : -73.49017333984375
G loss : 1977.060302734375
D loss : -804.263916015625
G loss : 5871.1025390625
D loss : -403.4736328125
G loss : 4048.26220703125
D loss : -757.6598510742188
G loss : 485.8670349121094
D loss : -777.1880493164062
G loss : 7467.7880859375
D loss : 939.97900390625
G loss : 771.2498779296875
D loss : -1013.253662109375
G loss : 261.740478515625
D loss : -1541.4971923828125
G loss : -176.21456909179688
D loss : -1336.904541015625
G loss : -1570.4083251953125
D loss : -1324.010986328125
G loss : -2643.109375
D loss : -462.2

G loss : -383.64251708984375
D loss : -112.55299377441406
G loss : -114.2801284790039
D loss : 290.324951171875
G loss : 1621.5771484375
D loss : 226.58181762695312
G loss : 527.4932861328125
D loss : -28.561384201049805
G loss : 570.4849243164062
D loss : -117.63570404052734
G loss : -105.25091552734375
D loss : -101.36009216308594
G loss : -1098.8258056640625
D loss : -214.77972412109375
G loss : -2252.3271484375
D loss : 311.181640625
G loss : -86.42366790771484
D loss : -89.77806854248047
G loss : -1090.2037353515625
D loss : -87.9085464477539
G loss : -347.2591247558594
D loss : 141.65045166015625
G loss : 1053.6541748046875
D loss : 80.49502563476562
G loss : 763.9994506835938
D loss : -76.89898681640625
G loss : 516.0343017578125
D loss : -150.043212890625
G loss : 424.4954833984375
D loss : 55.411895751953125
G loss : 106.19691467285156
D loss : 49.21154022216797
G loss : -281.53302001953125
D loss : -94.76853942871094
G loss : -1051.942626953125
D loss : -153.79046630859375
G 

...

G loss : 25.92731285095215
D loss : 5.6257452964782715
G loss : 81.23878479003906
D loss : -2.46364688873291
G loss : 58.0369987487793
D loss : -4.806398868560791
G loss : 87.40766906738281
D loss : 4.408959865570068
G loss : 80.44500732421875
D loss : -6.643787384033203
G loss : 76.77291870117188
D loss : 65.24232482910156
G loss : 28.080928802490234
D loss : -12.609971046447754
G loss : 25.595386505126953
D loss : 15.754053115844727
G loss : 10.972257614135742
D loss : 16.661434173583984
G loss : -5.3960161209106445
D loss : 9.538330078125
G loss : -11.15496826171875
D loss : 6.886640548706055
G loss : -24.65973472595215
D loss : 3.9922544956207275
G loss : -31.84903335571289
D loss : 24.818572998046875
G loss : -9.269725799560547
D loss : 3.6775946617126465
G loss : -28.883808135986328
D loss : 4.277896881103516
G loss : -35.12690734863281
D loss : -8.954115867614746
G loss : -31.91912841796875
D loss : 13.764113426208496
G loss : -37.120208740234375
D loss : -0.8836863040924072
G l

G loss : -0.2040625810623169
D loss : -20.42047119140625
G loss : 13.157487869262695
D loss : -5.855370044708252
G loss : -33.95707321166992
D loss : -13.647833824157715
G loss : -71.7073745727539
D loss : -0.376284122467041
G loss : -104.90447998046875
D loss : 88.55048370361328
G loss : -3.3910164833068848
D loss : 15.632353782653809
G loss : 10.453695297241211
D loss : -2.8892836570739746
G loss : 24.90729522705078
D loss : 8.127488136291504
G loss : 39.837791442871094
D loss : 8.534549713134766
G loss : 23.27964973449707
D loss : 8.195483207702637
G loss : 11.47948932647705
D loss : -0.9082610607147217
G loss : 18.222309112548828
D loss : -1.0917713642120361
G loss : 42.70265197753906
D loss : -0.358734130859375
G loss : 55.431068420410156
D loss : -7.394010543823242
G loss : -11.27109146118164
D loss : 4.802519798278809
G loss : -13.800533294677734
D loss : 15.569290161132812
G loss : -14.353055000305176
D loss : 9.816995620727539
G loss : -37.37071228027344
D loss : 9.25389099121

G loss : 4.521638870239258
D loss : -7.229280471801758
G loss : -1.8487123250961304
D loss : 3.5820913314819336
G loss : -10.732182502746582
D loss : -7.318149566650391
G loss : -30.3214111328125
D loss : 0.3961186408996582
G loss : -36.82771682739258
D loss : 3.736847162246704
G loss : -58.464237213134766
D loss : -12.507347106933594
G loss : -41.26536560058594
D loss : -2.038855791091919
G loss : -70.33781433105469
D loss : 6.48420524597168
G loss : -29.79492950439453
D loss : -2.989250421524048
G loss : -51.59394073486328
D loss : 8.289804458618164
G loss : -27.10816192626953
D loss : -14.362749099731445
G loss : -29.602529525756836
D loss : 2.6210098266601562
G loss : 0.7531756162643433
D loss : -8.39979362487793
G loss : -5.634104251861572
D loss : 9.01761531829834
G loss : -21.863597869873047
D loss : 13.226332664489746
G loss : 12.107498168945312
D loss : 26.401838302612305
G loss : 16.50101089477539
D loss : 10.752228736877441
G loss : 5.871391296386719
D loss : 1.6062798500061

G loss : 22.395809173583984
D loss : 7.322012424468994
G loss : 27.8963623046875
D loss : 2.3696975708007812
G loss : 27.131542205810547
D loss : 5.520340442657471
G loss : 36.126197814941406
D loss : 0.14872503280639648
G loss : 44.35301208496094
D loss : 7.627083778381348
G loss : 27.0467529296875
D loss : 1.7328550815582275
G loss : 7.623753547668457
D loss : 0.7264492511749268
G loss : 9.67448902130127
D loss : 11.240660667419434
G loss : 2.150242328643799
D loss : -3.275885581970215
G loss : -12.565973281860352
D loss : 7.473092079162598
G loss : -12.25802993774414
D loss : -13.376855850219727
G loss : -73.99845886230469
D loss : -3.066014528274536
G loss : -28.929698944091797
D loss : 9.4657621383667
G loss : -50.33512496948242
D loss : 1.3236579895019531
G loss : -38.284507751464844
D loss : 14.231181144714355
G loss : -40.3182487487793
D loss : 7.451496601104736
G loss : -21.371971130371094
D loss : 4.895597457885742
G loss : -36.18830108642578
D loss : -0.5968186855316162
G lo

G loss : -474.6597900390625
D loss : 111.00074768066406
G loss : -162.03712463378906
D loss : 38.27460479736328
G loss : -99.48918151855469
D loss : 66.44329833984375
G loss : 1.686483383178711
D loss : 5.592984199523926
G loss : 27.381649017333984
D loss : -13.005224227905273
G loss : 45.50687026977539
D loss : 5.758084297180176
G loss : 62.34543228149414
D loss : 11.939002990722656
G loss : 3.2792165279388428
D loss : -26.229461669921875
G loss : 296.8481750488281
D loss : -51.932945251464844
G loss : 534.7824096679688
D loss : -70.6947250366211
G loss : 104.77885437011719
D loss : -138.28897094726562
G loss : 427.52972412109375
D loss : 410.849853515625
G loss : 48.69613265991211
D loss : 67.04435729980469
G loss : 32.405338287353516
D loss : 50.94142532348633
G loss : 39.589298248291016
D loss : -55.37284851074219
G loss : 54.88957214355469
D loss : -3.661860466003418
